---
jupyter: python3
title: Python Google Drive API v3로 파일 업로드
toc: true
date: 2022-05-26
categories: [Python]
author: limyj0708
comments:
  giscus: 
    repo: quarto-dev/quarto-docs
---

### 1. google api 패키지 설치

In [ ]:
!pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

### 2. Credential 세팅 후 실행
1. [https://console.cloud.google.com/](https://console.cloud.google.com/) 접속
2. 원하는 프로젝트 선택
3. API & Services
   1. Enabled APIs & Services에서 Google Drive API 활성화
   2. Credentials
      1. Create Credentials -> OAuth client ID 생성
         - service account를 사용하고 싶었으나, 대상 폴더가 회사 조직 내 계정이 아니면 공유가 되지 않는 폴더여서 service account 사용이 불가능
         - 가능한 상황이면, service account를 대상 폴더의 편집자로 추가하는 편이, 더 보안상 좋다.
      2. Download OAuth Client
         - clinet-secret JSON 파일이 받아진다.
4. [https://developers.google.com/drive/api/quickstart/python](https://developers.google.com/drive/api/quickstart/python)
   1. quickstart 스크립트를 적절하게 바꾸어서 실행한다.
   2. 최초로 실행하면 로그인 과정 후에 token.json이 생성되고, 이후에는 token.json을 읽어서 실행된다.
   3. 아래 스크립트는 xlsx 파일 하나를 원하는 폴더에 업로드 하는 스크립트이다.

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
# 
SCOPES = ['https://www.googleapis.com/auth/drive.file']


def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('Download OAuth Clinet에서 받은 client-secret JSON 파일', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        # 원하는 작업 코드 작성
        # 이 경우에는, xlsx 파일 하나를 원하는 폴더에 업로드        
        folder_id = '원하는 폴더 ID'
        service = build('drive', 'v3', credentials=creds)
        file_metadata = {'name': 'quest_main_join_string_name.xlsx','parents': [folder_id]}
        media = MediaFileUpload('quest_main_join_string_name.xlsx',
                            mimetype=None, resumable=True)
        # 파일이 커질 것 같으면 resumable을 켜 주는 것이 좋다.
        file = service.files().create(body=file_metadata,media_body=media,fields='id').execute
        
        except HttpError as error:
            # TODO(developer) - Handle errors from drive API.
            print(f'An error occurred: {error}')

if __name__ == '__main__':
    main()